In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

ImportError: cannot import name 'Agent' from partially initialized module 'zero_liftsim.agent' (most likely due to a circular import) (/home/zero/code-repos/zero-lift-simulator/zero_liftsim/agent.py)

# Run Simulation

In [ ]:
cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 25, "lift_capacity": 4})
cfg['Lift']['num_chairs'] = 35

manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60*4)
print(result)

In [ ]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

## Sample Agent and Get Exp Traceback

In [ ]:
def sample_agent(manager, log, exp):
    # sample agent and subset exp/log to e/l for agent subset of those
    m = log['event'] == 'ride_complete'
    event_ride_comp = log[m].sample().iloc[0].to_dict()
    agent = manager.lookup_agent(event_ride_comp['agent_uuid'])
    l = log[log['agent_uuid'] == agent.agent_uuid]
    idx = (exp['time'] - event_ride_comp['time']).dt.total_seconds().abs().idxmin()
    e = exp.loc[idx].to_dict()
    return agent, l, e

In [ ]:
agent, l, e = sample_agent(manager, log, exp)

# Agent State Identification (ride loop exp)

for the ride loop agent experience, the total time is broken down into riding the lift, traversing down the mountain and waiting in the queue, i.e. "time_spent_total": ride + traverse + queue below:

sample a random time within the time range of the simulation:

In [ ]:
# sample a random timestamp within the agent log timeframe
INDEX_TIME = pd.Series(pd.date_range(log['time'].min(), log['time'].max(), freq='1s')).sample().iloc[0]

In [ ]:
s = inspect.getsource(AgentRideLoopExperience)
lines = s.split('\n')
idx = [i for i,x in enumerate(lines) if x.strip() == '"""'][-1]
print('\n'.join(lines[idx+1:]))

...and we can see in the source code above how `time_spent_total` is constructed. 

In [ ]:
from zero_liftsim.agent_state_id import infer_agent_states

# func 
r = infer_agent_states([agent], INDEX_TIME)

i = agent.agent_uuid
msg = 'func infer state should return agent uuid'
assert i in r, msg
print(r)

# Sample and Manually Review Examples

We need to evaluate how well the simulation is representing the actual constrained behavior that we see in the real world. Simply manually reviewing some examples here helps with that need. 

In [ ]:
def sample_time(log):
    return pd.Series(pd.date_range(log['time'].min(), log['time'].max(), freq='1s')).sample().iloc[0]

In [ ]:
time = sample_time(log)
agent = sample_agent(manager, log, exp)

In [ ]:
agent, l, e = sample_agent(manager, log, exp)